## Watsonx Lab 2

### Prompt Engineering guidelines

https://www.coursera.org/


In [1]:
!pip install ibm-watsonx-ai

!pip install ibm-watsonx-ai

In [2]:
!pip install python-dotenv

In [3]:
from ibm_watsonx_ai import APIClient
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import ModelInference
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get the API key from the environment variable
api_key = os.getenv('API_KEY')
# Get the project ID from the environment variable
project_id = os.getenv('PROJECT_ID')

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=api_key
)

client = APIClient(credentials)

In [4]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

model = ModelInference(
  model_id="ibm/granite-13b-chat-v2",
  # model_id= "mistralai/mistral-large",
  api_client=client,
  project_id=project_id,
  params = {
      "max_new_tokens": 100
  }
)

prompt = 'How far is Denver from Bangalore?'
print(model.generate(prompt))
print(model.generate_text(prompt))

/Users/jportilla/anaconda3/envs/3.11/lib/python3.11/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:389: LifecycleWarning: Model 'ibm/granite-13b-chat-v2' is in deprecated state from 2024-11-04 until 2025-02-03. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(


{'model_id': 'ibm/granite-13b-chat-v2', 'model_version': '2.1.0', 'created_at': '2024-10-30T15:10:24.529Z', 'results': [{'generated_text': "\nThe distance between Bangalore and Denver is approximately 13360 miles. The estimated flight time is around 22 hours and 30 minutes. However, it's important to note that this is a direct flight, and the actual travel time may vary depending on factors such as wind speed, air traffic, and the specific aircraft used.\n\nTo break down the distance further, the straight-line distance between Bangalore and Denver is about 9,000 miles, but the actual distance traveled by the aircraft will be", 'generated_token_count': 100, 'input_token_count': 8, 'stop_reason': 'max_tokens'}]}

The distance between Bangalore and Denver is approximately 13360 miles. The estimated flight time is around 22 hours and 30 minutes. However, it's important to note that this is a direct flight, and the actual travel time may vary depending on factors such as wind speed, air tra

In [5]:
#function to create access token
def get_access_token(api_key):
    import requests

    # Obtain an access token using the API key
    auth_url = "https://iam.cloud.ibm.com/identity/token"
    auth_headers = {
        "Content-Type": "application/x-www-form-urlencoded",
        "Accept": "application/json"
    }
    auth_data = {
        "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
        "apikey": api_key
    }

    auth_response = requests.post(auth_url, headers=auth_headers, data=auth_data)
    if auth_response.status_code != 200:
        raise Exception(f"Failed to obtain access token: {auth_response.status_code} - {auth_response.text}")

    access_token = auth_response.json().get('access_token')

    # Ensure the access token is available
    if not access_token:
        raise ValueError("Failed to obtain access token.")
    
    return access_token


In [6]:
def get_headers(api_key):
    access_token = get_access_token(api_key)
    headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": f"Bearer {access_token}"
    }
    return headers


In [7]:
#function to show results
def show_results(response):
    print(response.json())
    # Attempt to parse the response as JSON
    try:
        response_json = response.json()
    except requests.exceptions.JSONDecodeError:
        raise Exception(f"Failed to decode JSON response: {response.text}")

    # Extract and print the generated text from the results
    results = response_json.get('results', [])
    if results:
        generated_text = results[0].get('generated_text', 'No generated text found')
        print(generated_text)
    else:
        print("No results found in the response.")

# IBM Watsonx  summarization example

https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-prompt-samples.html?context=wx&audience=wdp#summarization

In [12]:
# API request
import  requests
import json
import os

url = "https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2023-05-29"
model_id = "google/flan-ul2"

# Get the API key & Project ID from the environment variables
api_key = os.getenv('API_KEY')
project_id = os.getenv('PROJECT_ID')


body = {
    "input": """The following document is a transcript from a financial earnings call.  Read the document and then write a short 1 paragraph summary.

Document: Financial Highlights

I’ll start with the financial highlights of the fourth quarter. We delivered $8 billion in revenue, $1.5 billion of operating pre-tax income and operating earnings per share of $1.5. In our seasonally strongest quarter, we generated $2.5 billion of free cash flow. Our revenue for the quarter was up over three percent at constant currency. While the dollar weakened a bit from 90 days ago, it still impacted our reported revenue by over $500 million – and 3 points of growth.  Revenue growth this quarter was again broad based. Product Sales revenue was up eight percent and Services up nine percent. These are our growth vectors and represent over 35 percent of our revenue. Within each of these segments, our growth was pervasive. We also had good growth across our geographies, with mid-single digit growth or better in Americas, EMEA and Asia Pacific. And for the year, we gained share overall. We had strong transactional growth in Product Sales to close the year. At the same time, our recurring revenue, which provides a solid base of revenue and profit, also grew.  Earnings Prepared RemarksEarnings were up 8.5 percent from last year. Looking at our profit metrics for the quarter, we expanded operating pretax margin by 85 basis points. This reflects a strong portfolio mix and improving Product and Consulting margins. These same dynamics drove a 30-basis point increase in operating gross margin. Our expense was down year to year, driven by currency dynamics. Within our base expense, the work we’re doing to digitally transform our operations provides flexibility to continue to invest in innovation and in talent. Our operating tax rate was 7 percent, which is flat versus last year. And our operating earnings per share of $1.5 was up over seven percent. Turning to free cash flow, we generated $2.5 billion in the quarter and $4.5 billion for the year. Our full-year free cash flow is up $1 billion fromlast year. As we talked about all year, we have a few drivers of our free cash flow growth. We had working capital improvements driven by efficiencies in ouroperations. Despite strong collections, the combination of revenue performance above our model and the timing of the transactions in the quarter led to higher-than-expected working capital at the end of the year. This impacted our free cash flow performance versus expectations. Our year-to-year free cash flow growthalso includes a modest tailwind from cash tax payments and lower payments for structural actions, partially offset by increased capex investment.

4Q Earnings Prepared RemarksWe ended the year in a strong liquidity position with cash of $4 billion. This is up over half a billion dollars year to year. And our debt balance is down nearly half a billion dollars. Our balance sheet remains strong, and I’d say the same for our retirement-related plans. At year end, our worldwide tax-qualified plansare funded at 57 percent, with the U.S. at 54 percent. Both are up year to year.  We transferred a portion of our U.S. qualified defined benefit plan obligations to insurers, without changing the benefits payable to plan participants. This resulted in a significant non-cash charge in our GAAP results in the third quarter, and we’ll see a benefit in our non-operating charges going forward.
Summary:""",
    "parameters": {
        "decoding_method": "greedy",
        "max_new_tokens": 300,
        "min_new_tokens": 50,
        "repetition_penalty": 2
    },
    "model_id": f"{model_id}",
    "project_id": f"{project_id}",
    "moderations": {
        "hap": {
            "input": {
                "enabled": True,
            }
        }
    }
}


response = requests.post(
    url,
    headers=get_headers(api_key),
    json=body
)

# Check for non-200 response and raise an exception if found
if response.status_code != 200:
    raise Exception(f"Non-200 response: {response.status_code} - {response.text}")


print("results: \n")
show_results(response)


results: 

{'model_id': 'google/flan-ul2', 'created_at': '2024-10-30T15:20:08.199Z', 'results': [{'generated_text': 'Revenue up 3% at constant currency. Operating pretax margin expanded by 85 basis points. Operating earnings per share of $1.5 was up over 7%. Free cash flow generated $2.5 billion in the quarter and $4.5 billion for the year. Full-year free cash flow is up $1 billion from last year.', 'generated_token_count': 64, 'input_token_count': 750, 'stop_reason': 'eos_token', 'seed': 0}], 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.', 'id': 'disclaimer_warning', 'more_info': 'https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx'}]}}
Revenue up 3% at constant currency. Operating pretax margin expanded by 85 basis points. Operating earnings per share of $1

## Prompting Principles
- **Principle 1: Write clear and specific instructions**
- **Principle 2: Give the model time to “think”**

### Tactics

#### Tactic 1: Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`

In [15]:
import  requests
import json
import os

url = "https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2023-05-29"
model_id = "google/flan-ul2"
# Get the API key & Project ID from the environment variables
api_key = os.getenv('API_KEY')
project_id = os.getenv('PROJECT_ID')





text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""


body = {
    "input": f"{prompt}",
    "parameters": {
        "decoding_method": "greedy",
        "max_new_tokens": 300,
        "min_new_tokens": 50,
        "repetition_penalty": 2
    },
    "model_id": f"{model_id}",
    "project_id": f"{project_id}",
    "moderations": {
        "hap": {
            "input": {
                "enabled": True,
            }
        }
    }
}


response = requests.post(
    url,
    headers=get_headers(api_key),
    json=body
)

# Check for non-200 response and raise an exception if found
if response.status_code != 200:
    raise Exception(f"Non-200 response: {response.status_code} - {response.text}")


print("results: \n")
show_results(response)

results: 

{'model_id': 'google/flan-ul2', 'created_at': '2024-10-30T15:40:19.577Z', 'results': [{'generated_text': "Write a clear and specific prompt. Don't confuse writing a  clear prompt with writing a short prompt. In many cases, longer prompts provide more clarity  and context for the model, which can lead to  more detailed and relevant outputs.", 'generated_token_count': 58, 'input_token_count': 135, 'stop_reason': 'eos_token', 'seed': 0}], 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.', 'id': 'disclaimer_warning', 'more_info': 'https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx'}]}}
Write a clear and specific prompt. Don't confuse writing a  clear prompt with writing a short prompt. In many cases, longer prompts provide more clarity  and context for t

#### Tactic 2: Ask for a structured output
- JSON, HTML

In [16]:
prompt = f"""
Generate a list of three made-up book titles along \ 
with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""


body = {
    "input": f"{prompt}",
    "parameters": {
        "decoding_method": "greedy",
        "max_new_tokens": 300,
        "min_new_tokens": 50,
        "repetition_penalty": 2
    },
    "model_id": f"{model_id}",
    "project_id": f"{project_id}",
    "moderations": {
        "hap": {
            "input": {
                "enabled": True,
            }
        }
    }
}


response = requests.post(
    url,
    headers=get_headers(api_key),
    json=body
)

# Check for non-200 response and raise an exception if found
if response.status_code != 200:
    raise Exception(f"Non-200 response: {response.status_code} - {response.text}")


print("results: \n")
show_results(response)

results: 

{'model_id': 'google/flan-ul2', 'created_at': '2024-10-30T15:45:31.367Z', 'results': [{'generated_text': "book_id, title, author, genre = ['The Book of the Dead', 'The Book of the Living', 'The Book of the Undead'];nnbook_id, title, author, genre = ['The Book of the Dead', 'The Book of the Living', 'The Book of the Undead'];n", 'generated_token_count': 91, 'input_token_count': 45, 'stop_reason': 'eos_token', 'seed': 0}], 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.', 'id': 'disclaimer_warning', 'more_info': 'https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx'}]}}
book_id, title, author, genre = ['The Book of the Dead', 'The Book of the Living', 'The Book of the Undead'];nnbook_id, title, author, genre = ['The Book of the Dead', 'The Book of the L

#### Tactic 3: Ask the model to check whether conditions are satisfied

In [17]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""

body = {
    "input": f"{prompt}",
    "parameters": {
        "decoding_method": "greedy",
        "max_new_tokens": 300,
        "min_new_tokens": 50,
        "repetition_penalty": 2
    },
    "model_id": f"{model_id}",
    "project_id": f"{project_id}",
    "moderations": {
        "hap": {
            "input": {
                "enabled": True,
            }
        }
    }
}


response = requests.post(
    url,
    headers=get_headers(api_key),
    json=body
)

# Check for non-200 response and raise an exception if found
if response.status_code != 200:
    raise Exception(f"Non-200 response: {response.status_code} - {response.text}")


print("results: \n")
show_results(response)

results: 

{'model_id': 'google/flan-ul2', 'created_at': '2024-10-30T15:47:36.808Z', 'results': [{'generated_text': 'Step 1 - Get some water boiling. Step 2 - Grab a cup and put a tea bag in it. Step 3 - Once the water is hot enough, just pour it over the tea bag. Step 4 - Let it sit for a bit so the tea can steep. Step 5 - After a few minutes, take out the tea bag.', 'generated_token_count': 79, 'input_token_count': 211, 'stop_reason': 'eos_token', 'seed': 0}], 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.', 'id': 'disclaimer_warning', 'more_info': 'https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx'}]}}
Step 1 - Get some water boiling. Step 2 - Grab a cup and put a tea bag in it. Step 3 - Once the water is hot enough, just pour it over the tea bag. Step 4 

In [18]:
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \ 
walk in the park. The flowers are blooming, and the \ 
trees are swaying gently in the breeze. People \ 
are out and about, enjoying the lovely weather. \ 
Some are having picnics, while others are playing \ 
games or simply relaxing on the grass. It's a \ 
perfect day to spend time outdoors and appreciate the \ 
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""


### complete cell as above


#### Tactic 4: "Few-shot" prompting

In [19]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""

### complete cell as above

### Principle 2: Give the model time to “think” 

#### Tactic 1: Specify the steps required to complete a task

In [20]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""

### Complete as above

#### Ask for output in a specified format

In [21]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""

### Complete as above

#### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

In [22]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""

### complete as above

#### Note that the student's solution is actually not correct.
#### We can fix this by instructing the model to work out its own solution first.

In [23]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""

### complete as above

## Model Limitations: Hallucinations
- Boie is a real company, the product name is not real.

In [24]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""

### complete as above

## Try experimenting on your own!

#### A note about the backslash
- In the course, we are using a backslash `\` to make the text fit on the screen without inserting newline '\n' characters.
- GPT-3 isn't really affected whether you insert newline characters or not.  But when working with LLMs in general, you may consider whether newline characters in your prompt may affect the model's performance.

# Summarizing
In this lesson, you will summarize text with a focus on specific topics.

## Setup

## Text to summarize

In [1]:
prod_review = """
Got this panda plush toy for my daughter's birthday, \
who loves it and takes it everywhere. It's soft and \ 
super cute, and its face has a friendly look. It's \ 
a bit small for what I paid though. I think there \ 
might be other options that are bigger for the \ 
same price. It arrived a day earlier than expected, \ 
so I got to play with it myself before I gave it \ 
to her.
"""

## Summarize with a word/sentence/character limit

In [2]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site to give feedback to the \
Shipping deparmtment. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words, and focusing on any aspects \
that mention shipping and delivery of the product. 

Review: ```{prod_review}```
"""

### complete as before

## Summarize with a focus on price and value

In [3]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site to give feedback to the \
pricing deparmtment, responsible for determining the \
price of the product.  

Summarize the review below, delimited by triple 
backticks, in at most 30 words, and focusing on any aspects \
that are relevant to the price and perceived value. 

Review: ```{prod_review}```
"""

### complete as above

#### Comment
- Summaries include topics that are not related to the topic of focus.

## Try "extract" instead of "summarize"

In [4]:

review_1 = prod_review 

# review for a standing lamp
review_2 = """
Needed a nice lamp for my bedroom, and this one \
had additional storage and not too high of a price \
point. Got it fast - arrived in 2 days. The string \
to the lamp broke during the transit and the company \
happily sent over a new one. Came within a few days \
as well. It was easy to put together. Then I had a \
missing part, so I contacted their support and they \
very quickly got me the missing piece! Seems to me \
to be a great company that cares about their customers \
and products. 
"""

# review for an electric toothbrush
review_3 = """
My dental hygienist recommended an electric toothbrush, \
which is why I got this. The battery life seems to be \
pretty impressive so far. After initial charging and \
leaving the charger plugged in for the first week to \
condition the battery, I've unplugged the charger and \
been using it for twice daily brushing for the last \
3 weeks all on the same charge. But the toothbrush head \
is too small. I’ve seen baby toothbrushes bigger than \
this one. I wish the head was bigger with different \
length bristles to get between teeth better because \
this one doesn’t.  Overall if you can get this one \
around the $50 mark, it's a good deal. The manufactuer's \
replacements heads are pretty expensive, but you can \
get generic ones that're more reasonably priced. This \
toothbrush makes me feel like I've been to the dentist \
every day. My teeth feel sparkly clean! 
"""

# review for a blender
review_4 = """
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \ 
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \ 
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \ 
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

reviews = [review_1, review_2, review_3, review_4]

In [5]:
for i in range(len(reviews)):
    prompt = f"""
    Your task is to generate a short summary of a product \ 
    review from an ecommerce site. 

    Summarize the review below, delimited by triple \
    backticks in at most 20 words. 

    Review: ```{reviews[i]}```
    """

## Try experimenting on your own!